# Exemples d’utilisation de `vsme_extractor`

Ce notebook montre comment utiliser la librairie `vsme_extractor` (sans passer par la CLI).

## Pré-requis

- Avoir installé le package (depuis ce repo) :
  - `pip install .`
- Fournir au minimum la variable d’environnement `SCW_API_KEY` (via un `.env` ou directement dans l’environnement).

## Logging (optionnel)

Le logging peut être activé via variables d’environnement (opt-in) :

- `SME_LOG_LEVEL` = `DEBUG|INFO|WARNING|ERROR`
- `VSME_LOG_FILE` = chemin vers un fichier `.log` (optionnel)
- `VSME_LOG_STDOUT` = `1|0` (true/false, yes/no, on/off)


In [ ]:
from pathlib import Path
import sys

from dotenv import load_dotenv

# Garantit l'import de la version du repo local (évite d'importer un package installé ailleurs).
# Dans la plupart des cas, le répertoire courant est `exemples/` → le repo root est le parent.
_CWD = Path.cwd().resolve()
_REPO_ROOT = _CWD if (_CWD / "vsme_extractor").exists() else _CWD.parent
if str(_REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(_REPO_ROOT))

import vsme_extractor  # noqa: E402
from vsme_extractor import VSMExtractor  # noqa: E402
from vsme_extractor.logging_utils import configure_logging_from_env  # noqa: E402
from vsme_extractor.stats import count_filled_indicators  # noqa: E402

print("vsme_extractor chargé depuis :", Path(vsme_extractor.__file__).resolve())

load_dotenv()  # charge SCW_API_KEY si vous avez un fichier .env
configure_logging_from_env()  # n'active le logging que si des variables SME_/VSME_ sont définies

## 1) Extraction d’un PDF

Modifiez `pdf_path` pour pointer vers un PDF existant.


In [ ]:
pdf_path = Path("path/to/report.pdf")  # <-- à modifier
pdf_path.exists(), pdf_path

In [ ]:
extractor = VSMExtractor(
    top_k_snippets=6,
    temperature=0.2,
    max_tokens=512,
)

df, stats = extractor.extract_from_pdf(str(pdf_path))
df.head()

In [ ]:
out_path = pdf_path.with_suffix(".vsme.xlsx")
df.to_excel(out_path, index=False)

print("Export:", out_path)
print("Tokens input :", stats.total_input_tokens)
print("Tokens output:", stats.total_output_tokens)
print("Coût total   :", f"{stats.total_cost_eur:.4f} €")

## 2) Extraction en batch d’un dossier de PDFs

Modifiez `folder` pour pointer vers un dossier contenant des PDFs (`*.pdf`).


In [ ]:
folder = Path("path/to/folder_with_pdfs")  # <-- à modifier
pdfs = sorted(folder.glob("*.pdf"))
len(pdfs), pdfs[:3]

In [ ]:
extractor = VSMExtractor()

for pdf in pdfs:
    df, stats = extractor.extract_from_pdf(str(pdf))
    out_path = pdf.with_suffix(".vsme.xlsx")
    df.to_excel(out_path, index=False)
    print(pdf.name, "→", out_path.name, "| coût:", f"{stats.total_cost_eur:.4f} €")

## 3) Calcul de complétude sur un dossier de `.vsme.xlsx`

Modifiez `results_dir` pour pointer vers un dossier contenant des fichiers `*.vsme.xlsx`.


In [ ]:
results_dir = Path("path/to/results_dir")  # <-- à modifier
excel_files = sorted(results_dir.glob("*.vsme.xlsx"))
len(excel_files), [p.name for p in excel_files[:5]]

In [ ]:
df_stats = count_filled_indicators(results_dir)
df_stats.head()

In [ ]:
stats_path = results_dir / "stats_completude.xlsx"
df_stats.to_excel(stats_path, index=False)
print("Stats exportées:", stats_path)